<a href="https://colab.research.google.com/github/ZahranWidebot/open_universal_arabic_asr_leaderboard/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install speechbrain
!pip install Cython
!pip install packaging
!pip install huggingsound
#!pip install "nemo_toolkit[all]"
!pip install pyctcdecode
!pip install qwen-omni-utils
!pip install omnilingual-asr
!pip install clearvoice






In [ ]:
"""
!pip install "nemo_toolkit[all]==1.17.0"

!pip install pyctcdecode tqdm huggingsound transformers speechbrain

# Install PyTorch (Colab usually has a compatible version, but let's ensure it)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install NVIDIA NeMo for ASR
!pip install nemo-toolkit[all]
!pip install soundfile
!pip install torchcodec
!pip install "ffmpeg"
"""

In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.7 MB/s eta 0:00:00


In [ ]:
# Install PyTorch (Colab usually has a compatible version, but let's ensure it)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install NVIDIA NeMo for ASR
!pip install nemo-toolkit[all]
!pip install soundfile

In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1


In [ ]:
!git clone https://github.com/Natural-Language-Processing-Elm/open_universal_arabic_asr_leaderboard


In [5]:
import json
import time
import torch
from tqdm import tqdm
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


In [ ]:
import os
import json
import soundfile as sf
from datasets import load_dataset, Audio

# Load dataset without decoding audio
ds = load_dataset("UBC-NLP/Casablanca", "Egypt", streaming=False)

In [ ]:
ds

DatasetDict({
    validation: Dataset({
        features: ['audio', 'seg_id', 'transcription', 'gender', 'duration'],
        num_rows: 846
    })
    test: Dataset({
        features: ['audio', 'seg_id', 'transcription', 'gender', 'duration'],
        num_rows: 846
    })
})

In [ ]:
from datasets import load_dataset, Audio
import os, json, soundfile as sf

data_folder = "casablanca_test_audio"
os.makedirs(data_folder, exist_ok=True)

# Load dataset in streaming mode
ds = load_dataset("UBC-NLP/Casablanca", "Egypt", streaming=True)

def save_audio_files(split_name):
    manifest_path = f"{split_name}_manifest.json"
    split_folder = os.path.join(data_folder, split_name)
    os.makedirs(split_folder, exist_ok=True)

    split = ds[split_name]
    with open(manifest_path, "w", encoding="utf-8") as f:
        for example in split:
            # Load audio on-the-fly
            audio_path = example["audio"]["path"]  # path to local/streamed file
            audio_array, sample_rate = sf.read(audio_path)

            filename = f"{example['seg_id']}.wav"
            save_path = os.path.join(split_folder, filename)
            sf.write(save_path, audio_array, sample_rate)

            item = {
                "audio_filepath": "{data_folder}/" + filename,
                "duration": len(audio_array)/sample_rate,
                "text": example.get("transcription", "")
            }
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")
    print(f"✔ Saved {split_name} manifest: {manifest_path}")

# Save splits
save_audio_files("validation")
save_audio_files("test")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
import sys
%cd /content/open_universal_arabic_asr_leaderboard
sys.path.append("/content/open_universal_arabic_asr_leaderboard")

from models.whisper import run_whisper



/content/open_universal_arabic_asr_leaderboard


In [10]:
# Example usage of run_omnilingual

# Path to the OmniLingual model you want to use
# Options: "omniASR_CTC_300M", "omniASR_LLM_300M", "omniASR_CTC_1B", "omniASR_LLM_1B",
#          "omniASR_CTC_3B", "omniASR_LLM_3B", "omniASR_CTC_7B", "omniASR_LLM_7B"
model_id = "openai/whisper-large-v3-turbo"

# Path to the manifest you created earlier
data_manifest = "/content/test_manifest.json"

# The folder where your audio files are stored
data_folder = "/content/casablanca_test_audio/test"

# Path to save the output manifest (predictions)
output_manifest = "casablanca_test_predictions.json"

# Run the OmniLingual ASR inference
run_whisper(model_id, data_manifest, data_folder, output_manifest)

print(f"Inference complete! Predictions saved to {output_manifest}")


Device set to use cuda:0
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
0it [00:00, ?it/s]


ZeroDivisionError: division by zero